# para2json 微调模型本地评测

本 Notebook 用于快速测试微调后的 para2json（段落→JSON）模型在本仓库管线中的效果（不额外生成文档，仅代码与结果展示）。


In [16]:
import os; os.environ["FCPD_POST_STRICT"] = "0"

In [17]:
import os, yaml, pathlib

# 项目根路径
ROOT = pathlib.Path('/Users/zhaowenyuan/Projects/FCPDExtractor')
CFG_PATH = ROOT / 'config.yaml'
MODELS_DIR = ROOT / 'models'

# 读取配置（仅用于取 finetuned_trigger_name）
with open(CFG_PATH, 'r', encoding='utf-8') as f:
    cfg = yaml.safe_load(f) or {}
local_cfg = (cfg.get('local_model') or {})

# 固定对比的两个模型（不做动态探测）
FT_NAME = 'My_Finetuned_Model_Q4_K_M_Mixed.gguf'
BASE_NAME = 'meta-llama-3.1-8b-instruct-q4_k_m-2.gguf'

assert (MODELS_DIR / FT_NAME).exists(), f'未找到微调模型: {FT_NAME}'
assert (MODELS_DIR / BASE_NAME).exists(), f'未找到原始模型: {BASE_NAME}'

print('对比模型:')
print('- 微调:', FT_NAME)
print('- 原始:', BASE_NAME)


对比模型:
- 微调: My_Finetuned_Model_Q4_K_M_Mixed.gguf
- 原始: meta-llama-3.1-8b-instruct-q4_k_m-2.gguf


In [ ]:
from core.local_pipeline import LocalPipeline

# 关闭 Llama3.1 chat 模板，强制使用普通 prompt（与 gpt4all/gguf 最稳）
USE_PLAIN_PROMPT = True

class ManualPipeline(LocalPipeline):
    def _get_chat_prompt(self, system: str, user: str, context: str = "", stage: str = "filter") -> str:
        if USE_PLAIN_PROMPT:
            return self._create_prompt(system, user, context)
        return super()._get_chat_prompt(system, user, context, stage)

# 使用上一单元中固定的模型名称
assert (MODELS_DIR / FT_NAME).exists(), f'未找到微调模型: {FT_NAME}'
assert (MODELS_DIR / BASE_NAME).exists(), f'未找到原始模型: {BASE_NAME}'

pipeline_ft_manual = ManualPipeline(
    model_name=None,
    model_path=str(MODELS_DIR),
    summarize_model=FT_NAME,
    finetuned_trigger_name=local_cfg.get('finetuned_trigger_name', 'My_Finetuned_Model'),
)

pipeline_base_manual = ManualPipeline(
    model_name=None,
    model_path=str(MODELS_DIR),
    summarize_model=BASE_NAME,
    finetuned_trigger_name=local_cfg.get('finetuned_trigger_name', 'My_Finetuned_Model'),
)

print('微调模型(手动对比):', FT_NAME)
print('原始模型(手动对比):', BASE_NAME)


微调模型(手动对比): My_Finetuned_Model_Q4_K_M_Mixed.gguf
原始模型(手动对比): meta-llama-3.1-8b-instruct-q4_k_m-2.gguf


## 手动输入段落对比（仅 summarize）

请在下一个单元编辑 `PARAS` 列表，然后运行“运行两个模型，统计解析率并逐段对比”单元。


In [19]:
# 手动输入段落：编辑此列表
PARAS = [
    # 示例：请用你自己的段落替换/追加
    """
    Flow rate 0.1 mL/min, T=80 °C in a 0.5 mm coil; yield 82%.
    """,
]

print(f"段落数: {len(PARAS)}")


段落数: 1


In [20]:
# 仅运行 summarize（不保存文件），返回原始与解析后的JSON
import json as _json
import pandas as _pd

_def_skeleton = {
    "reaction_summary": {
        "reaction_type": None,
        "reactants": [],
        "products": [],
        "conditions": [],
        "reactor": {"type": None, "inner_diameter": None},
        "metrics": {"conversion": None, "yield": None, "selectivity": None, "unit": "%"}
    }
}

def run_summarize_in_memory(pipeline, paragraphs):
    df = _pd.DataFrame({"content": [str(p) for p in paragraphs]})
    out = pipeline.summarize_parameters(df)
    raws = [str(s) for s in out.get('summarized', [])]
    parsed = []
    for s in raws:
        t = s.strip().replace('```json','').replace('```','')
        l, r = t.find('{'), t.rfind('}')
        if l != -1 and r != -1 and r > l:
            t = t[l:r+1]
        try:
            obj = _json.loads(t)
            parsed.append(obj)
        except Exception:
            parsed.append(_def_skeleton)
    ok = sum(1 for x in parsed if isinstance(x, dict))
    return {"raw": raws, "parsed": parsed, "ok": ok, "total": len(raws)}


In [21]:
# 运行两个模型，统计解析率并逐段对比
ft_res  = run_summarize_in_memory(pipeline_ft_manual, PARAS)
base_res = run_summarize_in_memory(pipeline_base_manual, PARAS)

print(f"微调解析率: {ft_res['ok']}/{ft_res['total']}")
print(f"原始解析率: {base_res['ok']}/{base_res['total']}")

import itertools

def to_str(x):
    import json
    try:
        return json.dumps(x, ensure_ascii=False, indent=2)
    except Exception:
        return str(x)

for i, (r1, r2, p1, p2) in enumerate(zip(ft_res['raw'], base_res['raw'], ft_res['parsed'], base_res['parsed']), 1):
    print(f"===== 段落 {i} =====")
    print("— 微调原文:")
    print(r1)
    print("— 原始原文:")
    print(r2)
    print("— 微调解析:")
    print(to_str(p1))
    print("— 原始解析:")
    print(to_str(p2))
    print()


  🔎 Summarize统计 -> 段落来源列: content, 段落总数: 1, 解析成功: 1, 含省略号: 0, 信号=0条数: 0
  🔎 Summarize统计 -> 段落来源列: content, 段落总数: 1, 解析成功: 0, 含省略号: 0, 信号=0条数: 0
微调解析率: 1/1
原始解析率: 1/1
===== 段落 1 =====
— 微调原文:
{"reaction_summary": {"reaction_type": "polymerization", "reactants": [{"name": "BA", "role": "catalyst"}], "products": [{"name": null, "yield_optimal": 82.0, "unit": "%"}], "conditions": [{"type": "temperature", "value": "80 °C"}, {"type": "residence_time", "value": null}, {"type": "flow_rate_total", "value": "0.1 mL/min"}, {"type": "pressure", "value": null}], "reactor": {"type": "coil", "inner_diameter": "0.5 mm"}, "metrics": {"conversion": 82, "yield": 82.0, "selectivity": null, "unit": "%"}}}
— 原始原文:
{"reaction_summary": {"reaction_type": null, "reactants": [], "products": [], "conditions": [], "reactor": {"type": null, "inner_diameter": null}, "metrics": {"conversion": null, "yield": null, "selectivity": null, "unit": "%"}}}
— 微调解析:
{
  "reaction_summary": {
    "reaction_type": "polymerizati

In [22]:
# 通用任务直测：指定 GGUF（My_Finetuned_Model_Q4_K_M_para2json_mixed_final_robust.gguf）+ 纯 JSON 回答
from gpt4all import GPT4All
import json as _json, re as _re
from pathlib import Path

TEST_MODEL = "My_Finetuned_Model_Q4_K_M_para2json_mixed_final_robust.gguf"
model_path = (MODELS_DIR / TEST_MODEL)
print(f"准备加载: {model_path}")
if not model_path.exists():
    print(f"❗未找到 {TEST_MODEL}，请确认文件是否存在于 {MODELS_DIR}")
else:
    # 使用较大的上下文，禁用下载
    test_model = GPT4All(str(TEST_MODEL), model_path=str(MODELS_DIR), allow_download=False, n_ctx=2048)

    def _extract_json_max_block(s: str) -> str:
        s = s.strip().replace('```json','').replace('```','')
        # 取最大一段 {} 块
        start = None
        depth = 0
        best = ""
        for i, ch in enumerate(s):
            if ch == '{':
                if depth == 0:
                    start = i
                depth += 1
            elif ch == '}':
                if depth > 0:
                    depth -= 1
                    if depth == 0 and start is not None:
                        seg = s[start:i+1]
                        if len(seg) > len(best):
                            best = seg
                        start = None
        return best if best else s

    # 极简系统 + 用户指令（仅 JSON、以 { 开头）
    system = "You output ONLY valid JSON. Start with '{' and end with '}'. No explanations."
    user = (
        "请用一个JSON对象回答：\n"
        "- 字段: name, definition, mime_type, example\n"
        "- 仅根据常识简要定义 JSON (JavaScript Object Notation)，给出一个简单示例对象\n"
        "- 只输出 JSON，不要多余文字\n"
        "- 字段含义：name=\"JSON\"，definition=一句话定义，mime_type=\"application/json\"，example 为一个包含2-3个键值对的小对象\n"
    )

    prompt = (
        f"{system}\n\nUser:\n{user}\nAssistant:\n"  # Plain 模板
        '{ "name": "JSON", "definition": '
    )

    raw = test_model.generate(prompt=prompt, max_tokens=512, temp=0.0)
    # 裁剪并轻清洗
    jtxt = _extract_json_max_block(raw)
    jtxt = _re.sub(r'(?<!")(?P<num>\d+(?:\.\d+)?)\s*%', r'\g<num>', jtxt)

    print("--- 原始输出 ---")
    print(raw)
    print("\n--- 最大花括号块 ---")
    print(jtxt)

    try:
        parsed = _json.loads(jtxt)
        print("\n✅ JSON 解析成功：")
        print(_json.dumps(parsed, ensure_ascii=False, indent=2))
    except Exception as e:
        print(f"\n❌ JSON 解析失败: {e}")
        print("请尝试：\n- 确认 GGUF 合并了 LoRA 并与底模/Tokenizer 匹配\n- 降低指令长度；保持起笔 '{'；尝试将 temp=0.1 或增加 max_tokens=1024\n- 或改用 HF 权重直接推理验证微调有效性后再导出 GGUF")



准备加载: /Users/zhaowenyuan/Projects/FCPDExtractor/models/My_Finetuned_Model_Q4_K_M_para2json_mixed_final_robust.gguf
❗未找到 My_Finetuned_Model_Q4_K_M_para2json_mixed_final_robust.gguf，请确认文件是否存在于 /Users/zhaowenyuan/Projects/FCPDExtractor/models


In [23]:
# 微调直推（严格提示词+清洗+一次重试）：禁止省略号、允许无 role 化学名（role=null）、严格拷贝字符串值
import json as _json, re as _re

_strict_system = "You output ONLY valid JSON. No explanations, no markdown, no comments."
_strict_user = (
    "Only use THIS paragraph; output ONLY one JSON object.\n"
    "Keys: reaction_summary.reaction_type, reactants[{name,role}], "
    "products[{name,yield_optimal,unit}], conditions[{type,value}], "
    "reactor{type,inner_diameter}, metrics{conversion,yield,selectivity,unit}.\n"
    "Condition types: temperature,residence_time,flow_rate_total,pressure.\n"
    "Copy condition values verbatim (e.g., '160 °C','21 min'). No examples/explanations/markdown.\n"
    "Start with '{' and end with '}'."
)

# 解析失败时的二次更严格提示
_strict_user_retry = (
    "Return ONLY valid JSON for THIS paragraph. No examples, no explanations, no code fences.\n"
    "Start with '{' and end with '}'. If any extra text appears, the answer is invalid.\n"
    "Keys must be exactly: reaction_summary.reaction_type, reactants[{name,role}], "
    "products[{name,yield_optimal,unit}], conditions[{type,value}], reactor{type,inner_diameter}, "
    "metrics{conversion,yield,selectivity,unit}. Values not present must be null or empty arrays."
)

def _extract_json_block(s: str) -> str:
    s = s.strip().replace('```json','').replace('```','')
    l, r = s.find('{'), s.rfind('}')
    if l != -1 and r != -1 and r > l:
        return s[l:r+1]
    return s

# 轻清洗：去示例说明/百分号/条件键名大小写/省略号
_def_cond_key_map = {
    'Temperature': 'temperature',
    'Residence_time': 'residence_time',
    'Flow_rate_total': 'flow_rate_total',
    'Pressure': 'pressure',
}

def _sanitize_strict_json(txt: str) -> str:
    s = txt
    # 去示例段
    s = _re.sub(r'(?is)\bExample\s+Input:.*$', '', s).strip()
    s = _re.sub(r'(?is)\bExample\s+Output:.*$', '', s).strip()
    # 仅保留最大花括号块
    s = _extract_json_block(s)
    # 97% -> 97（仅未加引号的百分数）
    s = _re.sub(r'(?<!")(?P<num>\d+(?:\.\d+)?)\s*%', r'\g<num>', s)
    # 统一 conditions.type 键名大小写
    for k,v in _def_cond_key_map.items():
        s = s.replace(f'"{k}"', f'"{v}"')
    # 禁用省略号 → null
    s = _re.sub(r':\s*("?\.\.\."?|…)(\s*[,}\]])', r': null\2', s)
    return s

ft_direct_raw = []
ft_direct_parsed = []

for para in PARAS:
    ctx = str(para)
    # 强制使用 Llama3.1 chat 模板以贴近微调分布
    # prompt = pipeline_ft_manual._create_llama31_chat_prompt(_strict_system, _strict_user, ctx)
    # model = pipeline_ft_manual._get_stage_model('summarize')
    # raw = pipeline_ft_manual._safe_generate(model, prompt, max_tokens=1024, temp=0.2)

    prompt = pipeline_ft_manual._create_llama31_chat_prompt(_strict_system, _strict_user, ctx) + "{"
    model = pipeline_ft_manual._get_stage_model('summarize')
    raw = pipeline_ft_manual._safe_generate(model, prompt, max_tokens=2048, temp=0.1)
    
    jtxt = _extract_json_block(raw)
    cleaned = _sanitize_strict_json(jtxt)
    ft_direct_raw.append(cleaned)
    obj = None
    try:
        obj = _json.loads(cleaned)
    except Exception:
        # 二次更严格重试
        prompt2 = pipeline_ft_manual._create_llama31_chat_prompt(_strict_system, _strict_user_retry, ctx)
        raw2 = pipeline_ft_manual._safe_generate(model, prompt2, max_tokens=700, temp=0.0)
        cleaned2 = _sanitize_strict_json(raw2)
        try:
            obj = _json.loads(cleaned2)
            ft_direct_raw[-1] = cleaned2
        except Exception:
            obj = None
            ft_direct_raw[-1] = cleaned2
    ft_direct_parsed.append(obj)

print('微调直推完成。条目数:', len(ft_direct_parsed))
for i, (r, p) in enumerate(zip(ft_direct_raw, ft_direct_parsed), 1):
    print(f"===== 直推段落 {i} =====")
    print('— 清洗后JSON:')
    print(r)
    print('— 解析结果:')
    try:
        print(_json.dumps(p, ensure_ascii=False, indent=2))
    except Exception:
        print(p)


微调直推完成。条目数: 1
===== 直推段落 1 =====
— 清洗后JSON:
{"reaction_summary": {"reaction_type": "null"}, "reactants": [{"name": "null", "role": "catalyst"}], "products": [], "conditions": [{"type": "temperature", "value": "80 °C"}], "reactor": {"type": "coil", "inner_diameter": "0.5 mm"}, "metrics": {"conversion": null, "yield": 82, "selectivity": null, "unit": "%"}}
— 解析结果:
{
  "reaction_summary": {
    "reaction_type": "null"
  },
  "reactants": [
    {
      "name": "null",
      "role": "catalyst"
    }
  ],
  "products": [],
  "conditions": [
    {
      "type": "temperature",
      "value": "80 °C"
    }
  ],
  "reactor": {
    "type": "coil",
    "inner_diameter": "0.5 mm"
  },
  "metrics": {
    "conversion": null,
    "yield": 82,
    "selectivity": null,
    "unit": "%"
  }
}
